# Model Deployment to Sagemaker

This notebook can be used to deploy a container located in the Amazon ECR. This jupyter notebook does the following

1. Creates a client and runtime pointed to the service "sagemaker" and "sagemaker-runtime", and creates a session for the sagemaker.
    - Sagemaker session is just used to get the details on the account
2. Model deployment as a Serverless Inference at the Sagemaker
    - Create a Model
    - Create an Endpoint configuration
    - Create an Endpoint to deploy the model
---

Information on the model deployment scenario in this notebook

- The model and all the associated files are located in the container itself
- The trained model files are located at ``/opt/ml/models/`` in the container. This can be seen in the DockerFile. Hence, no S3 location was mentioned while creating the endpoint
- The main programs in the container are located at `/opt/program/`
- The ENTRYPOINT program is `serve` and it is run using python3 symbolic linked as python at `/usr/bin/python`
- The DockerFile is configured to have a ENTRYPOINT[], so the program serve will be the only one that is running when the container goes online
- The deployed endpoint uses more than one model to make the prediction. So it will return a json file containing predictions from each of the models. These predictions can be used in majority vote to determine the final prediction
- The role for sagemaker access to the S3 bucket is required even if the models in the container does not require access to the S3 buckets

---
---
**NOTE**: Before running this script please ensure the following
- The current script creates only the endpoint em-ErrorIdentification. The endpoints, endpoints configuration, and models are required to have unique name and the name in this notebook are hardcoded. So, please ensure to modify as required before running this script
- The current script does not create a role for the sagemaker access to S3, which is by default required even if we don't require S3 for model deployment. The S3 bucket here will be used for logs and so on. The role required for the access to the S3 bucket from the sagemaker must be created seperately from the console


In [1]:
import sys
import boto3
import sagemaker
import json

## Session Creation
- To remember connection parameters to the SageMaker. Performing SageMaker operations

In [2]:
# Boto3 clients and runtime to create Models, EPC and EP
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

# Create a sagemaker session
sess = sagemaker.Session()
# Role to use
role = "arn:aws:iam::615766237552:role/service-role/AmazonSageMaker-ExecutionRole-20220308T113369"

In [3]:
# Get account details
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

# Get container name
image_name = "em_error-identification"
deployment_container = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, image_name)

# Display the container name
sys.stdout.write(f"Container name - {deployment_container}\n")

Container name - 615766237552.dkr.ecr.us-east-1.amazonaws.com/em_error-identification:latest


## Model Creation

Creating the model based on the provided container name
- Set the name for the model - Unique name
- Name of the container to be used - Must have already been built and deployed to ECS
- Execution role for the sagemaker to access a bucket for outputs etc.,

In [4]:
model_name = "em-ErrorIdentification"
create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": deployment_container,
            "Mode": "SingleModel"
        }
    ],
    Tags=[
        {
            "Key": "Research",
            "Value": "EnergyMonitoring"
        }
    ],
    ExecutionRoleArn=role
)

In [5]:
# Print the model ARN
sys.stdout.write(f"Model Arn-> {create_model_response['ModelArn']}\n")

Model Arn-> arn:aws:sagemaker:us-east-1:615766237552:model/em-erroridentification


## Endpoint Config Creation

Endpoint Configuration creation that is used to deploy the models
- Serverless or dedicated servers
- Model hosted by this endpoint configuration

In [6]:
em_epc_name = "em-ErrorIdentification"

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=em_epc_name,
    ProductionVariants=[
        {
            "VariantName": "emMachineFaultsID",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 2048,
                "MaxConcurrency": 20
            },
        },
    ],
    Tags=[
        {
            "Key": "Research",
            "Value": "EnergyMonitoring"
        }
    ]
)


In [7]:
sys.stdout.write(f"Endpoint Configuration ARN -> {endpoint_config_response['EndpointConfigArn']}\n")

Endpoint Configuration ARN -> arn:aws:sagemaker:us-east-1:615766237552:endpoint-config/em-erroridentification


## Endpoint Creation

Creating endpoint to provision resources and deploy models
- Launches the ML model container
- Deploys the models
- Takes and responses to requests

In [8]:
endpoint_name = "em-ErrorIdentification"

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=em_epc_name,
    Tags=[
        {
            "Key": "Research",
            "Value": "EnergyMonitoring"
        }
    ]
)

In [9]:
sys.stdout.write(f"Endpoint ARN -> {create_endpoint_response['EndpointArn']}\n")

Endpoint ARN -> arn:aws:sagemaker:us-east-1:615766237552:endpoint/em-erroridentification


In [10]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


{'EndpointName': 'em-ErrorIdentification',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:615766237552:endpoint/em-erroridentification',
 'EndpointConfigName': 'em-ErrorIdentification',
 'ProductionVariants': [{'VariantName': 'emMachineFaultsID',
   'DeployedImages': [{'SpecifiedImage': '615766237552.dkr.ecr.us-east-1.amazonaws.com/em_error-identification:latest',
     'ResolvedImage': '615766237552.dkr.ecr.us-east-1.amazonaws.com/em_error-identification@sha256:16ad02433912108cc2ae903916efefcca04b89a979b0bfd7e8bd488be039b5dd',
     'ResolutionTime': datetime.datetime(2022, 3, 11, 3, 31, 12, 713000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 0,
   'CurrentServerlessConfig': {'MemorySizeInMB': 2048, 'MaxConcurrency': 20}}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2022, 3, 11, 3, 31, 8, 446000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 3, 11, 3, 34, 13, 703000, tzinfo=tzlocal()),
 'Respon

## Endpoint Invocation

Can be used to test the deployed endpoint
- Send the data in the containers required format
- Get and print the response

In [ ]:
prediction_data_json = {
    "Power1": "25.0771255493164,25.0881195068359,25.0288944244384,25.0322895050048,25.019567489624,25.0811939239501,25.0567817687988,25.0910015106201,25.0314922332763,25.0788097381591,25.081443786621,25.0401401519775,25.045877456665,25.0404243469238,25.0076160430908,25.0512199401855,25.0346946716308,25.0446586608886,25.0332565307617,25.0574035644531,25.0968132019042,25.0615787506103,25.0382041931152,25.0491008758544,25.1241950988769,25.0392436981201,25.0230178833007,25.0204353332519,25.0519752502441,25.0240173339843,25.0427665710449,24.9379405975341,25.0041046142578,25.0308799743652,25.0612926483154,25.0320014953613,25.0422706604003,25.0051803588867,25.0279579162597,25.0852909088134,25.0930938720703,25.0698528289794,25.0710582733154,25.0477352142333,25.0519065856933,25.0614280700683,25.0723838806152,25.0991764068603,25.0865650177001,25.1170673370361,25.0540256500244,25.0798549652099,25.0849704742431,25.0312309265136,25.1255378723144,25.0225257873535,25.0904903411865,25.0474052429199,25.1046848297119,25.0453262329101",
    "Power2": "31.8992080688476,31.9162158966064,31.8855876922607,31.9471054077148,31.8855876922607,31.9012584686279,31.8825664520263,31.9387416839599,31.8746852874755,31.933271408081,31.9368438720703,31.8919258117675,31.9045295715332,31.9106216430664,31.9170131683349,31.903356552124,31.8900718688964,31.9255504608154,31.8304023742675,31.8938426971435,31.9322490692138,31.9114322662353,31.9216175079345,31.8925113677978,31.9254417419433,31.8634414672851,31.868501663208,31.842809677124,31.8761043548583,31.8554763793945,31.9000663757324,31.8127727508544,31.8391723632812,31.8676662445068,31.9318351745605,31.9253101348876,31.8505706787109,31.8570499420166,31.8396873474121,31.9334602355957,31.9582366943359,31.9120349884033,31.9268856048583,31.885944366455,31.8756065368652,31.9106254577636,31.9443378448486,31.9296722412109,31.9462642669677,31.9642715454101,31.9194774627685,31.9688968658447,31.928295135498,31.8822593688964,31.9845104217529,31.9201679229736,31.9468593597412,31.9262733459472,31.9417457580566,31.9017276763916",
    "Power3": "4.43631792068481,4.46914052963256,4.49889326095581,4.47576570510864,4.48098373413085,4.46267509460449,4.4888916015625,4.46709489822387,4.48896265029907,4.48088979721069,4.48646450042724,4.49688625335693,4.47577905654907,4.48818778991699,4.49505233764648,4.48401880264282,4.47496557235717,4.46222162246704,4.48596429824829,4.48356628417968,4.43776416778564,4.49333810806274,4.46358346939086,4.44732761383056,4.46853160858154,4.46510124206542,4.46153497695922,4.47355222702026,4.47754192352294,4.45179033279418,4.45174121856689,4.46737670898437,4.48824024200439,4.46070623397827,4.45336961746215,4.46745491027832,4.45958042144775,4.45017194747924,4.47335958480834,4.47972631454467,4.47209072113037,4.44024038314819,4.46984672546386,4.47031164169311,4.46693468093872,4.46359395980834,4.45703506469726,4.49207496643066,4.47972965240478,4.46260356903076,4.4754581451416,4.48571872711181,4.46361780166625,4.46358728408813,4.44026327133178,4.46618747711181,4.46967458724975,4.45312690734863,4.46352005004882,4.49186754226684",
    "PowerReac1": "12.6310081481933,12.6003704071044,12.626088142395,12.6118364334106,12.600754737854,12.6049375534057,12.5836496353149,12.5718736648559,12.6209192276,12.6005430221557,12.5940160751342,12.5961284637451,12.6235446929931,12.6139478683471,12.593765258789,12.6259155273437,12.639214515686,12.6105241775512,12.6375427246093,12.5983228683471,12.5843744277954,12.6189603805541,12.6173477172851,12.6058444976806,12.5287609100341,12.5656070709228,12.5747299194335,12.5772218704223,12.5510540008544,12.5742120742797,12.531150817871,12.5696372985839,12.54110622406,12.5320310592651,12.4954071044921,12.543755531311,12.5553255081176,12.5467700958251,12.5669536590576,12.5884027481079,12.5990324020385,12.5955867767333,12.5890550613403,12.6358852386474,12.608036994934,12.614254951477,12.6160383224487,12.5917205810546,12.5994625091552,12.5974254608154,12.6258707046508,12.5617389678955,12.6087388992309,12.6419715881347,12.5593214035034,12.5923271179199,12.5806846618652,12.5707464218139,12.5527610778808,12.5961799621582",
    "PowerReac2": "39.9311790466308,39.9287719726562,39.8695182800292,39.8786811828613,39.8772850036621,39.9042053222656,39.880931854248,39.8821067810058,39.8870773315429,39.9076766967773,39.9209899902343,39.8854026794433,39.9302978515625,39.8955612182617,39.8938446044921,39.9267463684082,39.9002609252929,39.893569946289,39.8576545715332,39.8711700439453,39.9071006774902,39.9385108947753,39.9014854431152,39.8905296325683,39.8584518432617,39.865364074707,39.8664588928222,39.8195304870605,39.8679962158203,39.8187713623046,39.8531532287597,39.7884445190429,39.8024559020996,39.7947082519531,39.8215446472167,39.8447456359863,39.8464927673339,39.7959365844726,39.8151016235351,39.9136848449707,39.8989448547363,39.8647727966308,39.9199714660644,39.88037109375,39.906925201416,39.8942260742187,39.9020118713378,39.957592010498,39.8847541809082,39.9177665710449,39.8540916442871,39.9601325988769,39.9330024719238,39.8640441894531,39.9342956542968,39.8636512756347,39.8578453063964,39.8496589660644,39.9173965454101,39.8476829528808",
    "PowerReac3": "32.273811340332,32.2627258300781,32.2629966735839,32.2315139770507,32.272403717041,32.2753829956054,32.2231903076171,32.2395248413085,32.2565879821777,32.2536506652832,32.2575798034667,32.268798828125,32.2644348144531,32.2631340026855,32.2549247741699,32.2451515197753,32.2756614685058,32.2449645996093,32.2379570007324,32.216697692871,32.2192192077636,32.2279815673828,32.240478515625,32.236473083496,32.1880187988281,32.201057434082,32.1856880187988,32.1719436645507,32.1713562011718,32.189525604248,32.1868095397949,32.1763954162597,32.1565170288085,32.1375846862792,32.1595191955566,32.1616783142089,32.1839942932128,32.1579818725585,32.1699638366699,32.2421264648437,32.2497062683105,32.2514381408691,32.2480506896972,32.231575012207,32.2493247985839,32.2715606689453,32.2155227661132,32.2609481811523,32.2356605529785,32.2600784301757,32.2805099487304,32.2405166625976,32.2581062316894,32.2314949035644,32.2361145019531,32.2285728454589,32.2262344360351,32.2102584838867,32.2403717041015,32.2389755249023",
    "PowerApp1": "31.9973392486572,32.0628471374511,31.9167232513427,31.9635925292968,31.8993301391601,32.0104026794433,31.993028640747,32.0442390441894,31.9228458404541,32.0052413940429,32.0328102111816,31.9092998504638,31.9205226898193,31.9534130096435,31.8907680511474,31.9378967285156,31.9194564819335,31.939697265625,31.8910655975341,31.9867210388183,32.0011749267578,31.9814624786376,31.882303237915,31.9415893554687,32.089298248291,31.9319229125976,31.8676509857177,31.872745513916,31.9383544921875,31.8822078704833,31.9228420257568,31.7282314300537,31.8679161071777,31.8592052459716,31.9510593414306,31.86669921875,31.9162559509277,31.767858505249,31.8577785491943,32.0154685974121,32.0070457458496,31.9278812408447,31.930736541748,31.8842620849609,31.9044361114501,31.900686264038,31.9745197296142,31.9964752197265,31.9795093536376,31.9887561798095,31.8736267089843,31.9711875915527,31.9327583312988,31.8239555358886,32.0448875427246,31.8227291107177,31.9926471710205,31.8485641479492,31.9568252563476,31.8712768554687",
    "Power2App": "53.3761978149414,53.4446029663085,53.3062934875488,53.3880386352539,53.3124160766601,53.3778495788574,53.3593101501464,53.406867980957,53.3186264038085,53.4180221557617,53.4258117675781,53.3221168518066,53.3641738891601,53.3718032836914,53.3465003967285,53.3539123535156,53.3419113159179,53.3578567504882,53.2612113952636,53.3438987731933,53.3781661987304,53.4018745422363,53.336139678955,53.3373756408691,53.4027824401855,53.3036231994628,53.2761535644531,53.2468070983886,53.3064193725585,53.2336921691894,53.3133583068847,53.1619224548339,53.2431297302246,53.2423515319824,53.3460540771484,53.3083877563476,53.276195526123,53.1740913391113,53.2337760925292,53.4264755249023,53.4070930480957,53.3101577758789,53.3613662719726,53.2945137023925,53.307331085205,53.3164100646972,53.3682823181152,53.4146461486816,53.392234802246,53.4100875854492,53.3097457885742,53.455078125,53.3605766296386,53.2576446533203,53.4583511352539,53.2774696350097,53.3723411560058,53.2976875305175,53.3784675598144,53.2853851318359",
    "Power3App": "32.574104309082,32.5662994384765,32.5692100524902,32.5337409973144,32.5833740234375,32.5793800354003,32.5371551513671,32.5443878173828,32.5668296813964,32.5586357116699,32.5729675292968,32.5806236267089,32.5702056884765,32.5778121948242,32.5653762817382,32.5535240173339,32.5846710205078,32.5518951416015,32.5414199829101,32.5262756347656,32.52001953125,32.5395965576171,32.5394668579101,32.535472869873,32.4955406188964,32.5042381286621,32.4962158203125,32.4825019836425,32.4831085205078,32.4943084716796,32.4886779785156,32.4820480346679,32.4714088439941,32.4429130554199,32.4659538269042,32.4707717895507,32.4956855773925,32.4633712768554,32.4752922058105,32.5594139099121,32.5589981079101,32.5544853210449,32.5570983886718,32.5340919494628,32.5530967712402,32.5800857543945,32.5182723999023,32.5732154846191,32.5491523742675,32.5652427673339,32.5863609313964,32.5493354797363,32.5666923522949,32.5380668640136,32.5355491638183,32.5421981811523,32.5345954895019,32.5131645202636,32.5425109863281,32.5513763427734"
}

In [ ]:
start_time = time.time()
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(prediction_data_json),
    ContentType="application/json"
)
end_time = time.time()

print(f"Time taken for a prediction - {end_time - start_time}")

In [ ]:
print(response["Body"].read())